# Imports

In [2]:
import numpy as np 
from skimage import io
from skimage.io import imread
import matplotlib.pyplot as plt
from scipy.stats import entropy
import os
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import timeit
from scipy.spatial import distance_matrix
import os
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from collections import Counter
import matplotlib.pyplot as plt
from stellargraph.layer.gcn import GraphConvolution

In [4]:
label_matrix2 = []
DATADIR = "dataset"
#CATEGORIES = ["Class1","Class2","Class3"]
CATEGORIES = ["Class1"]
Mean_matrix = []
Mean_index_matrix = []
b = 0
a = 1
while b < a:
    for category in CATEGORIES:
        labels = [0,0,0]
        if category == "Class1":
            label = "Class1"
            labels[0] = 1
        elif category == "Class2":
            label = "Class2"
            labels[1] = 1
        else:
            label = "Class3"
            labels[2] = 1

        path = os.path.join(DATADIR,category)
        clss_name, _ = os.path.splitext(category)
        for img in os.listdir(path): 

            mean_pixl = []
            mtrx = []
            name, _ = os.path.splitext(img)          
            image= img_as_float(imread(os.path.join(path,img)))            
            X = image.shape[0]
            Y = image.shape[1]
            segments = slic(image , n_segments = 200, compactness = 10, sigma = 1)        

    ##########################################

            keys = np.unique(segments)
            N = len(keys)
            a = N
            features =  np.zeros((N,6))
            label_matrix = np.zeros((N,3))
            print("key",len(keys))
            for key in keys:
                result = np.where(segments == key)
                listOfIndices= list(zip(result[0], result[1]))
                Mean_index = list(np.mean(listOfIndices,axis=0))
                Mean_index_matrix.append(Mean_index)
                RGB_Mean = np.mean(image[np.where(segments == key)],axis=0)
                RGB_Std = np.std(image[np.where(segments == key)],axis=0)
                feature = [RGB_Mean[0],RGB_Mean[1],RGB_Mean[2],RGB_Std[0],RGB_Std[1],RGB_Std[2]]
                mean_pixl.append(Mean_index)
                label_matrix2.append(label)

                label_matrix[b]= labels
                features [b] = feature
                b += 1
                
F = len(features[0])
print(features.shape)
print(label_matrix.shape)
 
dist_mat = distance_matrix(Mean_index_matrix, Mean_index_matrix, p=2)


dist_mat = distance_matrix(Mean_index_matrix, Mean_index_matrix, p=2)
matrix = []
for i in range(0,dist_mat.shape[0]):
    indx = []
    sr = sorted(dist_mat[i])
    for j in range(0,3):
        inx = np.where(dist_mat[i]==sr[j])
        
        r = inx[0]
        
        indx.extend(r)

    matrix.append(indx)
    
matrix_new2 = []
for x in matrix:
    y = list(np.unique(x))
    matrix_new2.append(y)

mx = len(matrix_new2)
Adj = np.zeros((mx,mx))
l = len(matrix)
for i in range(0,l):
    matrix2 = matrix_new2[i]
    for j in range(0,len(matrix2)):
        matrix2_m = matrix2[j]
        Adj[i,matrix2_m] = 1
print("Adj",np.array(Adj).shape)

start = timeit.default_timer()
stop = timeit.default_timer()
print('Time: ', stop - start)

C:\Users\Sanaz\AppData\Local\Temp/ipykernel_7124/79560190.py:39: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  segments = slic(image , n_segments = 200, compactness = 10, sigma = 1)


key 73
(73, 6)
(73, 3)
Adj (73, 73)
Time:  0.00010772099994937889


In [5]:
def limit_data(label_matrix2,limit=20,val_num=30,test_num=23):
    '''
    Get the index of train, validation, and test data
    '''
    label_counter = dict((l, 0) for l in label_matrix2)
    train_idx = []

    for i in range(len(label_matrix2)):
        label = label_matrix2[i]
        if label_counter[label]<limit:
            #add the example to the training data
            train_idx.append(i)
            label_counter[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == limit for count in label_counter.values()):
            break
    
    #get the indices that do not go to traning data
    rest_idx = [x for x in range(len(label_matrix2)) if x not in train_idx]
    val_idx = rest_idx[:val_num]
    test_idx = rest_idx[val_num:(val_num+test_num)]
    return train_idx, val_idx,test_idx

train_idx,val_idx,test_idx = limit_data(label_matrix2)

#set the mask
train_mask = np.zeros((N,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((N,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((N,),dtype=bool)
test_mask[test_idx] = True
print(val_mask)

[False False False False False False False False False False False False
 False False False False False False False False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False False False False False False False False False False
 False False False False False False False False False False False False
 False]


In [6]:
channels = 16          
dropout = 0.5           
l2_reg = 5e-4           
learning_rate = 1e-2    
epochs = 200           
es_patience = 10       

X_in = Input(shape=(N,F ))
fltr_in = Input((N,N ))

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConvolution(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])

dropout_2 = Dropout(dropout)(graph_conv_1)
num_classes = 3
graph_conv_2 = GraphConvolution(num_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
    log_dir='./Tensorboard_GCN_cora',
)
callback_GCN = [tbCallBack_GCN]
val_mask = np.expand_dims(val_mask, 0)
print(val_mask.shape)
train_mask = np.expand_dims(train_mask, 0)

label_matrix= np.expand_dims(label_matrix, 0)
features= np.expand_dims(features, 0)
Adj= np.expand_dims(Adj, 0)
print(np.array(label_matrix).shape)
validation_data = ([features, Adj], label_matrix, val_mask)
model.fit([features, Adj],
          label_matrix,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=7,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True),
              tbCallBack_GCN
          ])

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 73, 6)]      0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 73, 6)        0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 73, 73)]     0                                            
__________________________________________________________________________________________________
graph_convolution (GraphConvolu (None, 73, 16)       96          dropout[0][0]                    
                                                                 input_2[0][0]         

1/1 [==============================] - 0s 119ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 47/200
1/1 [==============================] - 0s 130ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 48/200
1/1 [==============================] - 0s 107ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 49/200
1/1 [==============================] - 0s 99ms/step - loss: 0.0154 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 50/200
1/1 [==============================] - 0s 93ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 51/200
1/1 [==============================] - 0s 86ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 52/200
1/1 [==============================] - 0s 146ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 53/200
1/1 [==============================] - 0s 114ms/step - loss: 0.0036 - acc: 1.00

1/1 [==============================] - 0s 115ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 109/200
1/1 [==============================] - 0s 97ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 110/200
1/1 [==============================] - 0s 99ms/step - loss: 0.0086 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 111/200
1/1 [==============================] - 0s 146ms/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 112/200
1/1 [==============================] - 0s 106ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 113/200
1/1 [==============================] - 0s 107ms/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 114/200
1/1 [==============================] - 0s 125ms/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 115/200
1/1 [==============================] - 0s 109ms/step - loss: 0.0033 - a

Epoch 170/200
1/1 [==============================] - 0s 116ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 119ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 126ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 110ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 111ms/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 175/200
1/1 [==============================] - 0s 108ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 176/200
1/1 [==============================] - 0s 108ms/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 177/200
1/1 [==============================] - 0s 181ms/step - 